In [1]:
import sys
import os
sys.path.append(os.path.abspath('/Users/leef/flow'))

# the Experiment class is used for running simulations
from flow.core.experiment import Experiment

# the base network class
from flow.networks import Network

# all other imports are standard
from flow.core.params import VehicleParams, NetParams, InitialConfig, EnvParams, \
    SumoParams, SumoCarFollowingParams, InFlows, SumoLaneChangeParams, TrafficLightParams
from flow.envs.ring.accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.controllers import SimLaneChangeController, GridRouter

# create some default parameters parameters
env_params = EnvParams()

vehicles = VehicleParams()
vehicles.add(
    veh_id="human",
    routing_controller=(GridRouter, {}),
    lane_change_controller=(SimLaneChangeController, {}),
    car_following_params=SumoCarFollowingParams(
        min_gap=2.5,
        decel=7.5,  # avoid collisions at emergency stops
    ),
    lane_change_params=SumoLaneChangeParams(
            lane_change_mode=1621,
        ),
    num_vehicles=1 # there may be errors if you add more vehicles
)

inflow = InFlows()
# off ramp inflow
inflow.add(
    veh_type='human',
    edge='ramp',
    probability=0.1,
    departLane='free',
    departSpeed=20)

# other inflow edges
outer_edges = ['bot1_0', 'bot0_0', 'top1_2', 'top0_3', 'left2_0', 'left2_1', 'right0_0', 'right0_1']
for i in range(len(outer_edges)):
    inflow.add(
        veh_type='human',
        edge=outer_edges[i],
        probability=0.25,
        departLane='free',
        departSpeed=20)
   

/Users/leef/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departLane in InFlows is deprecated, use depart_lane instead.
  PendingDeprecationWarning
/Users/leef/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departSpeed in InFlows is deprecated, use depart_speed instead.
  PendingDeprecationWarning
/Users/leef/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departLane in InFlows is deprecated, use depart_lane instead.
  PendingDeprecationWarning
/Users/leef/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departSpeed in InFlows is deprecated, use depart_speed instead.
  PendingDeprecationWarning
/Users/leef/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departLane in InFlows is deprecated, use depart_lane instead.
  PendingDeprecationWarning
/Users/leef/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute depart

In [2]:
net_params = NetParams(
    inflows=inflow,
    # used when routes only allow straight forward in intersections
    # template=os.path.abspath('/Users/leef/flow/tutorials/off_ramp_grid.net.xml'),
    # used for all possible turns situation
    # this file will provide more possible turns so that there will be more phases when define traffic lights
    template=os.path.abspath('/Users/leef/flow/tutorials/off_ramp_grid_turn.net.xml'),
)

# specify the edges vehicles can originate on
initial_config = InitialConfig(
    edges_distribution=['bot1_0', 'bot0_0', 'top1_2', 'top0_3', 'left2_0', 'left2_1', 'right0_0', 'right0_1', 'ramp']
)

# whether to allow turns at intersections
ALLOW_TURNS = True

# initialize traffic lights, used when you want define your own traffic lights
tl_logic = TrafficLightParams(baseline=False) # To see static traffic lights in action, the `TrafficLightParams` object should be instantiated with `baseline=False`

# when use off_ramp_grid.net.xml file, you should use a phase state example as "GGGgrrrrGGGgrrrr"
# when use off_ramp_grid_turn.net.xml file, you should use a phase state example as "GGGggrrrrrGGGggrrrrr"
if ALLOW_TURNS:
    phases = [{
        "duration": "31",
        "minDur": "8",
        "maxDur": "45",
        # for actuated traffic lights, you can add these optional values below
        # "maxGap": int, describes the maximum time gap between successive vehicle sthat will cause the current phase to be prolonged
        # "detectorGap": int, determines the time distance between the (automatically generated) detector and the stop line in seconds
        # "showDetectors": bool, toggles whether or not detectors are shown in sumo-gui
        "state": "GGGggrrrrrGGGggrrrrr"
    }, {
        "duration": "6",
        "minDur": "3",
        "maxDur": "6",
        "state": "yyyyyrrrrryyyyyrrrrr"
    }, {
        "duration": "31",
        "minDur": "8",
        "maxDur": "45",
        "state": "rrrrrGGGggrrrrrGGGgg"
    }, {
        "duration": "6",
        "minDur": "3",
        "maxDur": "6",
        "state": "rrrrryyyyyrrrrryyyyy"
    }]
    tl_logic.add("center0", phases=phases, programID=1, tls_type="actuated")
    tl_logic.add("center1", phases=phases, programID=1, tls_type="actuated")
    tl_logic.add("center2", phases=phases, programID=1, tls_type="actuated")
    tl_logic.add("center3", phases=phases, programID=1, tls_type="actuated")
else:
    phases = [{
        "duration": "31",
        "minDur": "8",
        "maxDur": "45",
        # for actuated traffic lights, you can add these optional values below
        # "maxGap": int, describes the maximum time gap between successive vehicle sthat will cause the current phase to be prolonged
        # "detectorGap": int, determines the time distance between the (automatically generated) detector and the stop line in seconds
        # "showDetectors": bool, toggles whether or not detectors are shown in sumo-gui
        "state": "GGGgrrrrGGGgrrrr"
    }, {
        "duration": "6",
        "minDur": "3",
        "maxDur": "6",
        "state": "yyyyrrrryyyyrrrr"
    }, {
        "duration": "31",
        "minDur": "8",
        "maxDur": "45",
        "state": "rrrrGGGgrrrrGGGg"
    }, {
        "duration": "6",
        "minDur": "3",
        "maxDur": "6",
        "state": "rrrryyyyrrrryyyy"
    }]

    # THIS IS A BUG THAT I DON'T KNOW WHY IT HAPPENS!!!!!!
    phase0 = [{
        "duration": "31",
        "minDur": "8",
        "maxDur": "45",
        "state": "GGrrGGrrGGrrGGrr"
    }, {
        "duration": "6",
        "minDur": "3",
        "maxDur": "6",
        "state": "yyrryyrryyrryyrr"
    }, {
        "duration": "31",
        "minDur": "8",
        "maxDur": "45",
        "state": "rrGGrrGGrrGGrrGG"
    }, {
        "duration": "6",
        "minDur": "3",
        "maxDur": "6",
        "state": "rryyrryyrryyrryy"
    }]

    tl_logic.add("center0", phases=phase0, programID=1, tls_type="actuated")
    tl_logic.add("center1", phases=phases, programID=1, tls_type="actuated")
    tl_logic.add("center2", phases=phases, programID=1, tls_type="actuated")
    tl_logic.add("center3", phases=phases, programID=1, tls_type="actuated")

    
    
# specify the routes for vehicles in the network
class offRampGrid(Network):
    def specify_routes(self, net_params):
        """See parent class."""
        
        if ALLOW_TURNS:    
            # allow all turns at intersections, but delete some routes
            return {'bot0_0': [
                        (['bot0_0', 'bot0_1', 'bot0_2', 'bot0_3'], 0.5), # wait for rewrite turns
                        (['bot0_0', 'left0_0'], 0.25),
                        (['bot0_0', 'right1_0', 'top1_0'], 0.1),
                        (['bot0_0', 'right1_0', 'right2_0'], 0.15),],
                    'top0_3': [
                        (['top0_3', 'top0_2', 'top0_1', 'top0_0'], 0.5), # wait for rewrite turns
                        (['top0_3', 'left0_1'], 0.25),
                        (['top0_3', 'right1_1', 'bot1_2'], 0.1),
                        (['top0_3', 'right1_1', 'right2_1'], 0.15),],
                    'bot1_0': [
                        (['bot1_0', 'bot1_1', 'bot1_2'], 0.5), # wait for rewrite turns
                        (['bot1_0', 'right2_0'], 0.25),
                        (['bot1_0', 'left1_0', 'top0_0'], 0.1),
                        (['bot1_0', 'left1_0', 'left0_0'], 0.15),],
                    'top1_2': [
                        (['top1_2', 'top1_1', 'top1_0'], 0.5), # wait for rewrite turns
                        (['top1_2', 'right2_1'], 0.25),
                        (['top1_2', 'left1_1', 'bot0_3'], 0.1),
                        (['top1_2', 'left1_1', 'left0_1'], 0.15),],
                    'left2_0': [
                        (['left2_0', 'left1_0', 'left0_0'], 0.5), # wait for rewrite turns
                        (['left2_0', 'top1_0'], 0.25),
                        (['left2_0', 'bot1_1', 'right2_1'], 0.1),
                        (['left2_0', 'bot1_1', 'bot1_2'], 0.15),],
                    'right0_0': [
                        (['right0_0', 'right1_0', 'right2_0'], 0.5), # wait for rewrite turns
                        (['right0_0', 'top0_0'], 0.25),
                        (['right0_0', 'bot0_1', 'bot0_2', 'left0_1'], 0.1),
                        (['right0_0', 'bot0_1', 'bot0_2', 'bot0_3'], 0.15),],
                    'left2_1': [
                        (['left2_1', 'left1_1', 'left0_1'], 0.5), # wait for rewrite turns
                        (['left2_1', 'top1_1', 'top1_0'], 0.15),
                        (['left2_1', 'top1_1', 'right2_0'], 0.1),
                        (['left2_1', 'bot1_2'], 0.25),],
                    'right0_1': [
                        (['right0_1', 'right1_1', 'right2_1'], 0.5), # wait for rewrite turns
                        (['right0_1', 'bot0_3'], 0.25),
                        (['right0_1', 'top0_2', 'top0_1', 'left0_0'], 0.1),
                        (['right0_1', 'top0_2', 'top0_1', 'top0_0'], 0.15),],

                    'ramp': [
                        (['ramp', 'bot0_2', 'bot0_3'], 0.5),
                        (['ramp', 'bot0_2', 'left0_1'], 0.25),
                        (['ramp', 'bot0_2', 'right1_1', 'bot1_2'], 0.1),
                        (['ramp', 'bot0_2', 'right1_1', 'right2_1'], 0.15),]}
        else:          
            # only allow straight forward in intersections
            return {'bot0_0': ['bot0_0', 'bot0_1', 'bot0_2', 'bot0_3'],
                      'top0_3': ['top0_3', 'top0_2', 'top0_1', 'top0_0'],
                      'bot1_0': ['bot1_0', 'bot1_1', 'bot1_2'],
                      'top1_2': ['top1_2', 'top1_1', 'top1_0'],
                      'left2_0': ['left2_0', 'left1_0', 'left0_0'],
                      'right0_0': ['right0_0', 'right1_0', 'right2_0'],
                      'left2_1': ['left2_1', 'left1_1', 'left0_1'],
                      'right0_1': ['right0_1', 'right1_1', 'right2_1'],

                      'ramp': ['ramp', 'bot0_2', 'bot0_3']}



flow_params = dict(
    exp_tag='A traffic light grid with exit ramp from highway',
    env_name=AccelEnv,
    network=offRampGrid,
    simulator='traci',
    sim=SumoParams(
        sim_step=0.1,
        render=True,
    ),
    env=EnvParams(
        horizon=1500,
        additional_params=ADDITIONAL_ENV_PARAMS.copy(),
    ),
    net=net_params,
    veh=vehicles,
    initial=initial_config,
    # used when you define your own traffic lights
    tls=tl_logic,
)

# number of time steps
flow_params['env'].horizon = 1000
exp = Experiment(flow_params)

# run the sumo simulation
_ = exp.run(1)

 Retrying in 1 seconds
Could not connect to TraCI server at localhost:59187 [Errno 61] Connection refused
 Retrying in 2 seconds
Could not connect to TraCI server at localhost:59187 [Errno 61] Connection refused
 Retrying in 3 seconds


KeyboardInterrupt: 